<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# Chapter 8: Baseline Experiments

**Purpose:** Train baseline models to understand data predictability and establish performance benchmarks.

**What you'll learn:**
- How to prepare data for ML with proper train/test splitting
- How to handle class imbalance with class weights
- How to evaluate models with appropriate metrics (not just accuracy!)
- How to interpret feature importance

**Outputs:**
- Baseline model performance (AUC, Precision, Recall, F1)
- Feature importance rankings
- ROC and Precision-Recall curves
- Performance benchmarks for comparison

---

## Evaluation Metrics for Imbalanced Data

| Metric | What It Measures | When to Use |
|--------|-----------------|-------------|
| **AUC-ROC** | Ranking quality across thresholds | General model comparison |
| **Precision** | "Of predicted churned, how many are correct?" | When false positives are costly |
| **Recall** | "Of actual churned, how many did we catch?" | When missing churners is costly |
| **F1-Score** | Balance of precision and recall | When both matter equally |
| **PR-AUC** | Precision-Recall under curve | Better for imbalanced data |

## 8.1 Setup

In [1]:
from customer_retention.analysis.auto_explorer import ExplorationFindings
from customer_retention.analysis.visualization import ChartBuilder, display_figure, display_table
from customer_retention.core.config.column_config import ColumnType
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_auc_score, classification_report, confusion_matrix,
                             roc_curve, precision_recall_curve, average_precision_score,
                             f1_score, precision_score, recall_score)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
# === CONFIGURATION ===
from pathlib import Path

FINDINGS_DIR = Path("../experiments/findings")

findings_files = [f for f in FINDINGS_DIR.glob("*_findings.yaml") if "multi_dataset" not in f.name]
if not findings_files:
    raise FileNotFoundError(f"No findings files found in {FINDINGS_DIR}. Run notebook 01 first.")

findings_files.sort(key=lambda f: f.stat().st_mtime, reverse=True)
FINDINGS_PATH = str(findings_files[0])

print(f"Found {len(findings_files)} findings file(s)")
print(f"Using: {FINDINGS_PATH}")

findings = ExplorationFindings.load(FINDINGS_PATH)

# Load data with snapshot preference (uses temporal snapshots if available)
from customer_retention.stages.temporal import load_data_with_snapshot_preference, TEMPORAL_METADATA_COLS
df, data_source = load_data_with_snapshot_preference(findings, output_dir="../experiments/findings")
charts = ChartBuilder()

print(f"\nLoaded {len(df):,} rows from: {data_source}")

FileNotFoundError: No findings files found in ../experiments/findings. Run notebook 01 first.

## 8.2 Prepare Data for Modeling

**📖 Feature Source:**

Features used in this notebook come from the **ExplorationFindings** generated in earlier notebooks:
- Column types are **auto-detected** in notebook 01 (Data Discovery)
- Target column is identified from the findings
- Identifier columns are **excluded** to prevent data leakage
- Text columns are **excluded** (require specialized NLP processing)

**📖 Best Practices:**
1. **Stratified Split**: Maintains class ratios in train/test sets
2. **Scale After Split**: Fit scaler on train only (prevents data leakage)
3. **Handle Missing**: Impute or drop before scaling

**📖 Transformations Applied:**
- Categorical variables → Label Encoded
- Missing values → Median (numeric) or Mode (categorical)
- Features → StandardScaler (fit on train only)

In [ ]:
if not findings.target_column:
    raise ValueError("No target column set. Please define one in exploration notebooks.")

target = findings.target_column
y = df[target]

# Features are selected based on column types from ExplorationFindings
# This ensures consistency with earlier notebooks and prevents data leakage
feature_cols = [
    name for name, col in findings.columns.items()
    if col.inferred_type not in [ColumnType.IDENTIFIER, ColumnType.TARGET, ColumnType.TEXT]
    and name not in TEMPORAL_METADATA_COLS  # Exclude point-in-time columns from features
]

print("=" * 70)
print("FEATURE SELECTION FROM FINDINGS")
print("=" * 70)
print(f"\n🎯 Target Column: {target}")
print(f"📊 Features Selected: {len(feature_cols)}")

# Show feature breakdown by type
type_counts = {}
for name in feature_cols:
    col_type = findings.columns[name].inferred_type.value
    type_counts[col_type] = type_counts.get(col_type, 0) + 1

print("\n📋 Features by Type:")
for col_type, count in sorted(type_counts.items()):
    print(f"   {col_type}: {count}")

# Show excluded columns
excluded = [name for name, col in findings.columns.items() 
            if col.inferred_type in [ColumnType.IDENTIFIER, ColumnType.TARGET, ColumnType.TEXT]]
if excluded:
    print(f"\n⛔ Excluded Columns: {', '.join(excluded)}")

In [ ]:
X = df[feature_cols].copy()

# Encode categorical variables
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Handle missing values (median for numeric, mode for others)
for col in X.columns:
    if X[col].isnull().any():
        if X[col].dtype in ['int64', 'float64']:
            X[col] = X[col].fillna(X[col].median())
        else:
            X[col] = X[col].fillna(X[col].mode()[0])

# Stratified train/test split (maintains class distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features (fit on train only!)
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print(f"Train size: {len(X_train):,} ({len(X_train)/len(X)*100:.0f}%)")
print(f"Test size: {len(X_test):,} ({len(X_test)/len(X)*100:.0f}%)")
print(f"\nTrain class distribution:")
print(f"  Retained (1): {(y_train == 1).sum():,} ({(y_train == 1).sum()/len(y_train)*100:.1f}%)")
print(f"  Churned (0): {(y_train == 0).sum():,} ({(y_train == 0).sum()/len(y_train)*100:.1f}%)")

## 8.3 Baseline Models (with Class Weights)

**📖 Using Class Weights:**
- `class_weight='balanced'` automatically adjusts weights inversely proportional to class frequencies
- This helps models pay more attention to the minority class (churned customers)
- Without weights, models may just predict "retained" for everyone

In [ ]:
# Models with class_weight='balanced' to handle imbalance
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced'),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42)
}

results = []
model_predictions = {}

for name, model in models.items():
    print(f"Training {name}...")
    
    # Use scaled data for Logistic Regression, unscaled for tree-based
    if "Logistic" in name:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    auc = roc_auc_score(y_test, y_pred_proba)
    pr_auc = average_precision_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # Cross-validation
    if "Logistic" in name:
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='roc_auc')
    else:
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
    
    results.append({
        "Model": name,
        "Test AUC": auc,
        "PR-AUC": pr_auc,
        "F1-Score": f1,
        "Precision": precision,
        "Recall": recall,
        "CV AUC Mean": cv_scores.mean(),
        "CV AUC Std": cv_scores.std()
    })
    
    model_predictions[name] = {
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'model': model
    }

results_df = pd.DataFrame(results)
results_df = results_df.round(4)

print("\n" + "=" * 80)
print("MODEL COMPARISON")
print("=" * 80)
display_table(results_df)

## 8.4 Feature Importance (Random Forest)

In [ ]:
rf_model = models["Random Forest"]
importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": rf_model.feature_importances_
}).sort_values("Importance", ascending=False)

top_n = 15
top_features = importance_df.head(top_n)

fig = charts.bar_chart(
    top_features["Feature"].tolist(),
    top_features["Importance"].tolist(),
    title=f"Top {top_n} Feature Importances"
)
display_figure(fig)

## 8.5 Classification Report (Best Model)

In [ ]:
best_model = models["Gradient Boosting"]
y_pred = best_model.predict(X_test)

print("Classification Report (Gradient Boosting):")
print(classification_report(y_test, y_pred))

## 8.6 Model Comparison Grid

This visualization shows all models side-by-side with:
- **Row 1**: Confusion matrices (counts and percentages)
- **Row 2**: ROC curves with AUC scores
- **Row 3**: Precision-Recall curves with PR-AUC scores

**📖 How to Read:**
- **Confusion Matrix**: Diagonal = correct predictions. Off-diagonal = errors.
- **ROC Curve**: Higher curve = better. AUC > 0.8 is good, > 0.9 is excellent.
- **PR Curve**: Higher curve = better at finding positives without false alarms.

In [ ]:
# Create comprehensive model comparison grid
# Uses the framework's ChartBuilder.model_comparison_grid method

# Prepare model results in the expected format
grid_results = {
    name: {
        "y_pred": data["y_pred"],
        "y_pred_proba": data["y_pred_proba"]
    }
    for name, data in model_predictions.items()
}

# Create the visualization grid
fig = charts.model_comparison_grid(
    grid_results,
    y_test,
    class_labels=["Churned (0)", "Retained (1)"],
    title="Model Comparison: Confusion Matrix | ROC Curve | Precision-Recall"
)

display_figure(fig)

# Summary metrics table
print("\n" + "=" * 80)
print("METRICS SUMMARY")
print("=" * 80)
display_table(results_df[["Model", "Test AUC", "PR-AUC", "F1-Score", "Precision", "Recall"]])

### 8.6.1 Individual Model Analysis

The grid above shows all models together. Below is detailed analysis per model.

In [ ]:
# Individual model classification reports
print("=" * 70)
print("CLASSIFICATION REPORTS BY MODEL")
print("=" * 70)

for name, data in model_predictions.items():
    print(f"\n{'='*40}")
    print(f"📊 {name}")
    print('='*40)
    print(classification_report(y_test, data['y_pred'], target_names=["Churned", "Retained"]))

### 8.6.1 Precision-Recall Curves

**📖 Why PR Curves for Imbalanced Data:**
- ROC curves can look optimistic for imbalanced data
- PR curves focus on the minority class (churners)
- Better at showing how well we detect actual churners

**📖 How to Read:**
- **Baseline** (dashed line) = proportion of positives in the data
- Higher curve = better at finding churners without too many false alarms

## 8.7 Key Takeaways

**📖 Interpreting Results:**

In [ ]:
best_model = results_df.loc[results_df['Test AUC'].idxmax()]

print("=" * 70)
print("KEY TAKEAWAYS")
print("=" * 70)

print(f"\n🏆 BEST MODEL: {best_model['Model']}")
print(f"   Test AUC: {best_model['Test AUC']:.4f}")
print(f"   PR-AUC: {best_model['PR-AUC']:.4f}")
print(f"   F1-Score: {best_model['F1-Score']:.4f}")

print(f"\n📊 TOP 3 IMPORTANT FEATURES:")
for i, feat in enumerate(importance_df.head(3)['Feature'].tolist(), 1):
    imp = importance_df[importance_df['Feature'] == feat]['Importance'].values[0]
    print(f"   {i}. {feat} ({imp:.3f})")

print(f"\n📈 MODEL PERFORMANCE ASSESSMENT:")
if best_model['Test AUC'] > 0.90:
    print("   Excellent predictive signal - likely production-ready with tuning")
elif best_model['Test AUC'] > 0.80:
    print("   Strong predictive signal - good baseline for improvement")
elif best_model['Test AUC'] > 0.70:
    print("   Moderate signal - consider more feature engineering")
else:
    print("   Weak signal - may need more data or different features")

print(f"\n💡 NEXT STEPS:")
print("   1. Feature engineering with derived features (notebook 05)")
print("   2. Hyperparameter tuning (GridSearchCV)")
print("   3. Threshold optimization for business metrics")
print("   4. A/B testing in production")

---

## Summary: What We Learned

In this notebook, we trained baseline models and established performance benchmarks:

1. **Data Preparation** - Proper train/test split with stratification and scaling
2. **Class Imbalance Handling** - Used balanced class weights
3. **Model Comparison** - Compared Logistic Regression, Random Forest, and Gradient Boosting
4. **Multiple Metrics** - Evaluated with AUC, PR-AUC, F1, Precision, Recall
5. **Feature Importance** - Identified the most predictive features

## Key Results for This Dataset

| Metric | Value | Interpretation |
|--------|-------|----------------|
| Best AUC | ~0.98 | Excellent discrimination |
| Top Feature | esent | Email engagement is critical |
| Imbalance | ~4:1 | Moderate, handled with class weights |

---

## Next Steps

Continue to **09_business_alignment.ipynb** to:
- Align model performance with business objectives
- Define intervention strategies by risk level
- Calculate expected ROI from the model
- Set deployment requirements

In [ ]:
best_auc = max(float(r["Test AUC"]) for r in results)

print("Key Takeaways:")
print("="*50)
print(f"Best baseline AUC: {best_auc:.4f}")
print(f"Top 3 important features: {', '.join(importance_df.head(3)['Feature'].tolist())}")

if best_auc > 0.85:
    print("\nStrong predictive signal detected. Data is well-suited for modeling.")
elif best_auc > 0.70:
    print("\nModerate predictive signal. Consider feature engineering for improvement.")
else:
    print("\nWeak predictive signal. May need more features or data.")

---

## Next Steps

Continue to **09_business_alignment.ipynb** to align with business objectives.